In [ ]:
import pandas as pd
from prophet import Prophet
import numpy as np
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(0)

In [ ]:
def create_train_test(stock):
    stock = stock.rename(columns={'Date':'ds', stock.columns[1]:'y'})


    train_size = int(len(stock) * 0.8)
    test_size = len(stock) - train_size
    train, test = stock[:train_size], stock[train_size:]
    
    return stock, train, test

In [ ]:
def prophet_model(changepoint_prior_scale=0.05, seasonality_prior_scale=10., interval_width= 0.8):
    model = Prophet(changepoint_prior_scale=changepoint_prior_scale, 
                    seasonality_prior_scale=seasonality_prior_scale,
                    interval_width=interval_width)
    model.fit(train)
    future = model.make_future_dataframe(periods=1444, freq ='d')
    future = future[future['ds'].isin( stock['ds'] )]
    forecast = model.predict(future)

    fig = model.plot(forecast)
    ax = fig.gca()
    ax.plot(stock["ds"], stock["y"])
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.set_title('')

    print("Train - Root Mean Squared Error (RMSE):", math.sqrt(mean_squared_error(y_true = train["y"], y_pred = forecast['yhat'][:len(train)])))
    print("Test - Root Mean Squared Error (RMSE):", math.sqrt(mean_squared_error(y_true = test["y"], y_pred = forecast['yhat'][len(train):])))

In [ ]:
## Load Data
df = pd.read_csv('Stocks Data.csv', header = 1)
df = df.iloc[1:, :].rename(columns={'Symbols': 'Date'}).reset_index().drop('index',axis=1)
df.Date = pd.to_datetime(df.Date)

In [ ]:
# Separate dataframe by stock
mrin = df[['Date', 'MRIN']].dropna()
mxl = df[['Date', 'MXL']].dropna()
agys = df[['Date', 'AGYS']]
dell = pd.read_csv('DELL.csv')[['Date', 'Close']]
aapl = pd.read_csv('AAPL.csv')[['Date', 'Close']]
dell.Date = pd.to_datetime(dell.Date)
aapl.Date = pd.to_datetime(aapl.Date)

stocks = [aapl, mrin, mxl, dell, agys]

In [ ]:
stock.head()

## MRIN

In [ ]:
stock, train, test = create_train_test(mrin)

In [ ]:
prophet_model(changepoint_prior_scale=0.5, seasonality_prior_scale=0.0001, interval_width=0.2)

## AAPL

In [ ]:
stock, train, test = create_train_test(aapl)

In [ ]:
prophet_model(changepoint_prior_scale=0.8, seasonality_prior_scale=0.1, interval_width=0.3)

## MXL

In [ ]:
stock, train, test = create_train_test(mxl)

In [ ]:
prophet_model(seasonality_prior_scale=0.00007)

## DELL

In [ ]:
stock, train, test = create_train_test(dell)

In [ ]:
prophet_model(changepoint_prior_scale=8, seasonality_prior_scale=0.4, interval_width=0.2)

## AGYS

In [ ]:
stock, train, test = create_train_test(agys)

In [ ]:
prophet_model(changepoint_prior_scale=9, seasonality_prior_scale=0.7, interval_width=0.2)